In [1]:
import os
import datetime
import pandas as pd
import re
import great_expectations
import agriculture as ag

In [2]:
#!pip install great_expectations

## Descargando los datos

Para descargar los datos referentes a los precios de diversos productos agricolas, podemos ejecutar directamente el script `agriculture.py` en la terminal de la siguiente manera:

```python3 agriculture.py```

El script mencionado llama de forma iterativa a la API de la pagina del Sistema Nacional de Informacion de Mercados (SNIIM) de la Secretaria de Economia, para recuperar los precios de diversos productos en diversos centros de distribucion en el pais, desde el año 2020 hasta la fecha.

Otra manera de descargar los datos es ejecutando la siguiente celda, la cual inicializa un objeto de tipo `ScrapperMarketAgriculture`, el cual contiene la funcion `scraping`, la cual realiza el mismo proceso del script mencionado anteriormente.

In [3]:
# ESTE SCRAPPER PUEDE TARDAR, RECOMENDABLE EJECUTAR DIRECTAMENTE DESDE TERMINAL
# scrapper = ag.ScrapperMarketAgriculture() # Generamos un objeto de tipo ScrapperMarketAgriculture (scrapper SNIIM)
# scrapper.scraping() # Scrapear la pagina del SNIIM para las frutas y hortalizas

Una vez recuperados los datos, estos se almacenan de forma local en archivos .csv, los cuales se encuentran en el directorio `data/sniim`

In [4]:
# Directorio que contiene los archivos generados por el scrapper
sniim_dir = "./data/sniim/"

## Procesamiento de los datos

Como al final tendremos una dataframe conteniendo los precios de diversos productos, podemos tratar de obtener el nombre de cada producto a partir del nombre del archivo csv, para esto definimos la siguiente funcion:

In [5]:
# Definimos una funcion para obtener el nombre del cultivo a partir del nombre del archivo csv
def estandarizar_nombre(nombre : str) -> str:
    """
    Regresa el nombre del cultivo/producto a partir del nombre del archivo .csv
    """
    cultivo_std = nombre.split('_')[:-1]
    cultivo_std = '_'.join(cultivo_std)
    cultivo_std = re.sub(r"(?<=#)_(?=\d)",'',cultivo_std)
    cultivo_std = re.sub("_"," ",cultivo_std)
    cultivo_std = re.sub("csemilla", "con semilla", cultivo_std)
    cultivo_std = re.sub("ssemilla", "sin semilla", cultivo_std)
    cultivo_std = cultivo_std.title()

    return cultivo_std

Finalmente, podemos iterar sobre los archivos presentes en el directorio anterior, para generar una dataframe por cada uno de ellos, agregar las dataframes a una lista para posteriormente concatenarlas en una sola.

Ademas, a cada dataframe le agregamos la columna `producto` para poder diferenciar las entradas una vez concatenadas las dataframes, y separamos la columna `destino` en dos: `destino` y `central` para identificar mas facilmente la Entidad de destino y la central de abastos donde se levanto la muestra.

In [6]:
# Almacenaremos nuestras dataframes en una lista para despues concatenar
df_list = []
for dirpath, subdir, files in os.walk(sniim_dir):
    for file in files:
        if file == ".DS_Store":
            continue
        
        df = pd.read_csv(os.path.join(dirpath,file)) 
        df["producto"] = estandarizar_nombre(file) # Obtenemos el nombre del producto a partir del nombre del archivo
        df[["destino","central"]] = df["destino"].str.split(": ", expand=True) # Separamos la columna destino en 2
        
        # Reordenamos las columnas
        cols = df.columns.to_list()
        del cols[-1]
        cols.insert(4,"central")
        
        del cols[-1]
        cols.insert(1,"producto")
        
        df = df[cols]
        
        # Agregamos la dataframe generada a la lista
        df_list.append(df)

In [7]:
precios_sniim = pd.concat(df_list,ignore_index=True) # Concatenamos nuestra lista de dataframes en una sola

Una vez obtenida nuestra dataframe final, podemos utilizar el metodo `sample` para obtener una vista general de nuestra dataframe.

In [8]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec,obs
604720,17/04/2020,Guanabana,Caja de 17 kg.,Michoacán,Jalisco,Mercado Felipe Ángeles de Guadalajara,22.94,22.94,22.94,NaN
883399,13/06/2022,Piña Chica,Pieza,Veracruz,México,Central de Abasto de Ecatepec,30.00,33.0,30.00,NaN
846794,19/08/2020,Uva Globo,Kilogramo,Distrito Federal,Chiapas,Central de Abasto de Tuxtla Gutiérrez,45.00,50.0,45.00,NaN
619106,20/01/2020,Tamarindo,Kilogramo,Michoacán,Michoacán,Mercado de Abasto de Morelia,25.00,33.00,33.00,NaN
280408,12/08/2020,Piña Mediana,Caja de 20 kg.,Jalisco,Chihuahua,Central de Abasto de Chihuahua,16.00,17.0,17.00,NaN
397377,07/05/2021,Platano Manzano,Kilogramo,Tabasco,Campeche,"Mercado ""Pedro Sáinz de Baranda"", Campeche",22.00,25.0,25.00,NaN
929544,21/09/2022,Tuna,Caja de 18 kg.,México,Morelos,Central de Abasto de Cuautla,11.11,12.22,11.11,NaN
180607,15/07/2020,Chayote Sin Espinas,Caja de 18 kg.,Distrito Federal,Campeche,"Mercado ""Pedro Sáinz de Baranda"", Campeche",13.33,13.33,13.33,NaN
883410,28/06/2022,Piña Chica,Pieza,Veracruz,México,Central de Abasto de Ecatepec,40.00,45.0,45.00,NaN
366297,19/05/2020,Chile Pimiento Morron,Caja de 10 kg.,Jalisco,Guanajuato,Central de Abasto de León,25.00,30.0,25.00,NaN


Para una mayor facilidad al momento de comprender nuestra dataframe, generamos un diccionario de datos para esta, explicando lo que representa cada una de las columnas.

In [9]:
precios_dict = {
    "fecha": "Fecha en la que se levanto la encuesta",
    "product": "Producto al que hace referencia la entrada/muestra",
    "presentacion": "Presentacion del producto al que se hace referencia",
    "origen": "Entidad Federativa de donde proviene el producto",
    "destino": "Entidad Federativa hacia donde llego el producto",
    "central": "Central de abastos/centro de distribucion de donde se obtuvo la informacion",
    "precio_min": "El valor más bajo de la cotización dentro de una muestra (MXN)",
    "precio_max": "El valor más alto de la cotización dentro de una muestra (MXN)",
    "precio_frec": "Es el dato que más se repite en la muestra (moda) (MXN)",
    "obs": "Observaciones encontradas para la muestra"
}

In [10]:
dict_df = pd.DataFrame.from_dict(precios_dict,orient="index")
dict_df

,0
fecha,Fecha en la que se levanto la encuesta
product,Producto al que hace referencia la entrada/mue...
presentacion,Presentacion del producto al que se hace refer...
origen,Entidad Federativa de donde proviene el producto
destino,Entidad Federativa hacia donde llego el producto
central,Central de abastos/centro de distribucion de d...
precio_min,El valor más bajo de la cotización dentro de u...
precio_max,El valor más alto de la cotización dentro de u...
precio_frec,Es el dato que más se repite en la muestra (mo...
obs,Observaciones encontradas para la muestra


## Limpieza de los datos

Una vez generado nuestro diccionario, podemos comenzar a tratar los datos de nuestro dataframe. Algo que podemos observar del sample de nuestro dataframe es que la columna `obs` parece ser la unica que cuenta con valores nulos. Podemos tener una idea de la proporcion de datos faltantes en esta columna ejecutando la siguiente celda, la cual calcula el porcentaje de entradas faltantes en la misma.

In [11]:
print(f"Porcentaje de entradas faltantes en columna 'obs': {100* precios_sniim['obs'].isna().sum() / precios_sniim.shape[0]:.2f}%")

Porcentaje de entradas faltantes en columna 'obs': 96.26%


Vemos que la mayoria de las entradas en nuestra dataframe (> 90%) no cuenta con informacion en la columna `obs`, por lo que puede ser mas conveniente deshacernos de esta columna.

In [12]:
precios_sniim.drop(columns="obs", inplace=True)

In [13]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
938804,27/04/2021,Lechuga Orejona Sin Clasificacion,Pieza,Distrito Federal,Yucatán,Centro Mayorista Oxkutzcab,14.00,14.0,14.00
608589,09/03/2022,Limon Con Semilla #4,Kilogramo,Sinaloa,Puebla,Central de Abasto de Puebla,40.00,50.0,45.00
501714,24/03/2022,Fresa Chandler,Caja de 4 kg.,Distrito Federal,Veracruz,Mercado Malibrán,87.50,112.5,105.00
917827,19/01/2023,Durazno Amarillo,Caja de 9 kg.,Michoacán,México,Central de Abasto de Toluca,38.89,44.44,38.89
471047,28/02/2022,Guanabana,Kilogramo,Nayarit,Jalisco,Mercado Felipe Ángeles de Guadalajara,27.50,27.5,27.50
324985,19/03/2020,Cebolla Bola,Kilogramo,Guanajuato,Coahuila,"Central de Abasto de La Laguna, Torreón",12.00,12.6,12.50
110960,21/05/2020,Col Grande,Kilogramo,Nayarit,Nayarit,Mercado de abasto 'Adolfo López Mateos' de Tepic,12.00,14.0,13.00
671490,04/05/2021,Limon Con Semilla #5,Caja de 19 kg.,Colima,Aguascalientes,Centro Comercial Agropecuario de Aguascalientes,14.21,15.26,14.74
793033,05/10/2022,Zanahoria Mediana,Arpilla de 24 kg.,Puebla,DF,Central de Abasto de Iztapalapa DF,6.25,7.08,6.67
33666,24/03/2020,Chile Poblano,Arpilla de 20 kg.,Sinaloa,DF,Central de Abasto de Iztapalapa DF,20.00,27.0,24.00


Posteriormente, podemos utilizar el metodo `info` de nuestra dataframe para obtener informacion acerca de los tipos de datos de las columnas o la posible presencia de mas datos faltantes.

In [14]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982933 entries, 0 to 982932
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   fecha         982933 non-null  object 
 1   producto      982933 non-null  object 
 2   presentacion  982933 non-null  object 
 3   origen        982933 non-null  object 
 4   destino       982933 non-null  object 
 5   central       982933 non-null  object 
 6   precio_min    982933 non-null  float64
 7   precio_max    982933 non-null  object 
 8   precio_frec   982933 non-null  float64
dtypes: float64(2), object(7)
memory usage: 67.5+ MB


Lo primero que notamos es que la columna `fecha` tiene un tipo de dato `object`, lo cual puede significar que esta almacenada como un string. Al tratarse de datos temporales, podemos convertir esta columna a un objeto de tipo `datetime` para una mayor facilidad al momento de manipular o explorar los datos.

In [15]:
precios_sniim["fecha"] = pd.to_datetime(precios_sniim["fecha"])

Despues podemos analizar los valores en la columna `origen` para verificar que no haya valores inesperados. Los posibles valores en esta columna son los nombres de los 32 estados de Mexico, o bien "Importacion" si el producto fue importado del extranjero, o "Nacional" si no hay datos sobre su estado de origen.

In [16]:
precios_sniim["origen"].unique()

array(['Nayarit', 'Distrito Federal', 'Oaxaca', 'Importación', 'Sonora',
       'Yucatán', 'Baja California', 'Campeche', 'San Luis Potosí',
       'Chiapas', 'Durango', 'Hidalgo', 'Michoacán', 'Jalisco',
       'Guanajuato', 'Sinaloa', 'Zacatecas', 'México', 'Tamaulipas',
       'Chihuahua', 'Quintana Roo', 'Puebla', 'Coahuila', 'Guerrero',
       'Colima', 'Veracruz', 'Nuevo León', 'Morelos', 'Querétaro',
       'Baja California Sur', 'Aguascalientes', 'Tabasco', 'Nacional'],
      dtype=object)

Vemos que no existen anomalias aparentes en esta columna, asi que podemos proceder a revisar los valores de la columna `destino`, la cual deberia tener valores similares a la columna anterior.

In [17]:
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California ', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Aguascalientes', 'Zacatecas',
       'Nuevo León', 'Sonora', 'Morelos', 'Quintana Roo', 'Hidalgo',
       'Puebla', 'San Luis Potosí', 'Baja California Sur', 'Veracruz',
       'Chihuahua', 'Tabasco', 'Colima', 'Sinaloa', 'Tamaulipas',
       'Querétaro'], dtype=object)

Vemos que tampoco hay anomalias en esta columna. Sin embargo, algunos valores contienen espacios al final del valor (por ejemplo, `Baja California `). Esto se puede corregir facilmente utilizando la funcion `strip` que nos ofrece pandas:

In [18]:
precios_sniim["destino"] = precios_sniim["destino"].str.strip()
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Aguascalientes', 'Zacatecas',
       'Nuevo León', 'Sonora', 'Morelos', 'Quintana Roo', 'Hidalgo',
       'Puebla', 'San Luis Potosí', 'Baja California Sur', 'Veracruz',
       'Chihuahua', 'Tabasco', 'Colima', 'Sinaloa', 'Tamaulipas',
       'Querétaro'], dtype=object)

Finalmente, la columna `precio_max` aparece como de tipo `object`, por lo que posiblemente hay valores que impidieron que fuera parseada a `float`.

Despues de una inspeccion de esta columna, vemos que para algunos precios, se utiliza la coma (,) para separar los millares. Podemos eliminar esta coma de nuestro dataframe con el metodo `replace`, para despues transformarla a tipo `float` con el metodo `astype(float)`.

In [19]:
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(str).str.replace(",","")
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(float)

Finalmente, podemos volver a obtener una muestra de nuestro dataframe para ver su estructura final, asi como volver a llamar al metodo `info` para verificar los tipos de datos de cada columna y que no haya valores faltantes.

In [20]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
68094,2020-05-25,Zanahoria Polvo,Arpilla de 23 kg.,Puebla,México,Central de Abasto de Toluca,4.78,6.09,5.22
798255,2022-07-29,Nopal,Caja de 20 kg.,Distrito Federal,Campeche,"Mercado ""Pedro Sáinz de Baranda"", Campeche",18.00,20.00,20.00
45064,2022-04-11,Cebolla De Rabo,Manojo de 5 kg.,Puebla,DF,Central de Abasto de Iztapalapa DF,9.00,11.00,10.00
405460,2023-05-19,Platano Manzano,Kilogramo,Yucatán,Quintana Roo,"Mercado de Chetumal, Quintana Roo",25.00,25.00,25.00
102426,2022-08-26,Pera Angel,Caja de 20 kg.,Importación,Baja California,"Central de Abasto INDIA, Tijuana",18.25,18.25,18.25
113435,2020-02-01,Melon Cantaloupe #36,Kilogramo,Puebla,Guerrero,Central de Abastos de Acapulco,16.00,16.00,16.00
303319,2021-03-08,Melon Cantaloupe #27,Caja de 38 kg.,Coahuila,DF,Central de Abasto de Iztapalapa DF,9.21,10.53,10.00
4816,2020-01-24,Uva Perlette,Caja de 8 kg.,Sonora,Jalisco,Mercado Felipe Ángeles de Guadalajara,37.50,37.50,37.50
982930,2022-11-01,Lechuga Orejona Grande,Pieza,Oaxaca,Oaxaca,Módulo de Abasto de Oaxaca,11.00,12.00,12.00
799155,2022-03-25,Nopal,Kilogramo,Nayarit,Nayarit,Mercado de abasto 'Adolfo López Mateos' de Tepic,27.00,29.00,28.00


In [21]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982933 entries, 0 to 982932
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   fecha         982933 non-null  datetime64[ns]
 1   producto      982933 non-null  object        
 2   presentacion  982933 non-null  object        
 3   origen        982933 non-null  object        
 4   destino       982933 non-null  object        
 5   central       982933 non-null  object        
 6   precio_min    982933 non-null  float64       
 7   precio_max    982933 non-null  float64       
 8   precio_frec   982933 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 67.5+ MB
